# Cleaning file
_This notebook is made by Katoo_

### Importing required libraries

In [15]:
import pandas as pd
import numpy as np
from scipy import stats
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import LabelEncoder

### Checking and dropping missing values

In [16]:
df = pd.read_csv('subset_2016.csv')
print(df.isna().sum())
df = df.dropna()

Transaction unique identifier        0
Price                                0
Date of Transfer                     0
Property Type                        0
Old/New                              0
Duration                             0
Town/City                            0
District                             0
County                               0
PPDCategory Type                     0
Record Status - monthly file only    0
dtype: int64


### Checking and dropping duplicates

In [17]:
# Check for duplicate rows in the dataset
duplicates = df[df.duplicated()]
print(f'Number of duplicate rows: {len(duplicates)}')
# Check for duplicates
df = df.drop_duplicates()


Number of duplicate rows: 0


### Checking for skewness in Price column
- skewness > 0: right (positive) skewness
- skewness < 0: left (negative) skewness 
- skewness = 0: symmetric distribution

Applying log transformations to reduce the right skewness (compressing large values )

In [18]:
# Check the current skewness
old_skew = df["Price"].skew()
print("Old skewness:", old_skew)

# Apply log transformation to correct right-skewness
df['Price'] = np.log(df["Price"])
new_skew = df["Price"].skew()
print("New skewness:", new_skew)

Old skewness: 45.26318224583534
New skewness: -0.12253996519765045


### Viewing the first few rows of the dataset to have a quick view on the contents of the dataframe


In [19]:
df.head()

,Transaction unique identifier,Price,Date of Transfer,Property Type,Old/New,Duration,Town/City,District,County,PPDCategory Type,Record Status - monthly file only
0,{31FB4C16-D962-57B9-E050-A8C063053436},14.436087,2016-03-21,F,N,L,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A
1,{31FB4C16-D963-57B9-E050-A8C063053436},12.860999,2016-04-15,F,N,L,LONDON,HAMMERSMITH AND FULHAM,GREATER LONDON,A,A
2,{31FB4C16-D964-57B9-E050-A8C063053436},13.199324,2016-04-21,F,N,L,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A
3,{31FB4C16-D965-57B9-E050-A8C063053436},14.648420,2016-03-31,T,N,F,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A
4,{31FB4C16-D966-57B9-E050-A8C063053436},14.790070,2016-03-31,F,N,L,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A


### Extracting year, month and day from the Date of Transfer column and dropping the original column

In [20]:
df['year'] = df['Date of Transfer'].str.split('-').str[0]
df['month'] = df['Date of Transfer'].str.split('-').str[1]
df['day'] = df['Date of Transfer'].str.split('-').str[2]
df.drop(columns='Date of Transfer', axis= 1, inplace=True)

### Renaming columns 
- converting to lowercase and replace spaces with underscores

In [21]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

### Converting categorical variables
- CategoricalDtype(): creates custom categorical type for the column
- .astype(categorical_type): converts column to the new categorical type

In [22]:
# Convert categorical variables
property_type = CategoricalDtype(categories=['D', 'S', 'T', 'F'], ordered=False)
df["property_type"] = df["property_type"].astype(property_type)
old_new = CategoricalDtype(categories=['Y', 'N'], ordered=False)
df["old/new"] = df["old/new"].astype(old_new)
duration = CategoricalDtype(categories=['F', 'L'], ordered=False)
df["duration"] = df["duration"].astype(duration)
ppdcategory_type = CategoricalDtype(categories=['A', 'B'], ordered=False)
df["ppdcategory_type"] = df["ppdcategory_type"].astype(ppdcategory_type)
record_status = CategoricalDtype(categories=['A', 'C', 'D'], ordered=False)
df["record_status_-_monthly_file_only"] = df["record_status_-_monthly_file_only"].astype(record_status)


### One-hot encoding for property_type and label encoding for old/new
- Convert categorical column property_type into multiple binary columns
- Ensure that one-hot encoded columns contain numeric values (0 and 1) instead of boolean values
Label Encoding for old/new
- Convert categorical column old/new into numeric labels (1 for 'Y', 0 for 'N').

In [23]:
# One-Hot Encoding for 'property_type'
df = pd.get_dummies(df, columns=['property_type'], prefix='property_type_is_', drop_first=False)
df[df.filter(like='property_type_is_').columns] = df.filter(like='property_type_is_').astype(int)

# Label Encoding for 'old/new'
encoder = LabelEncoder()
df['old/new'] = encoder.fit_transform(df['old/new'])


### Dropping unnecessary columns

In [24]:

df.drop(columns = 'transaction_unique_identifier', axis = 1, inplace = True)
df.drop(columns = 'duration', axis = 1, inplace = True)
df.drop(columns = 'ppdcategory_type', axis = 1, inplace = True)
df.drop(columns = 'record_status_-_monthly_file_only', axis = 1, inplace = True)

### Outlier detection and removel in the price column
- interquartile range method 
    - calculate IQR
    - set bounds
    - detect outliers
    - count outliers
    - clip outliers
    - verify clipping
- alternative IQR method (in comment)
    - instead of clipping data -> removing it
    - rows where price is outside the acceptable range are removed
    - count number of removed rows
- z-score method (in comment)
    - calculate z-score
    - identify outliers
    - filter data so only the entries that are not outliers are saved

In [25]:
# Remove outliers
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
higher = Q3 + 1.5 * IQR

price_outliers_below = df.loc[df['price'] < lower]
price_outliers_abow = df.loc[df['price'] > higher]

print(price_outliers_below['price'].count(), "entries having 'Price' value lower than ", lower)
print(price_outliers_abow['price'].count(), "entries having 'Price' value greater than", higher)

df["price"] = df["price"].clip(lower=lower, upper=higher)

# Identify outliers
price_outliers_below = df.loc[df['price'] < lower]
price_outliers_above = df.loc[df['price'] > higher]

# Print outlier counts
print(price_outliers_below['price'].count(), "entries having 'Price' value lower than", lower)
print(price_outliers_above['price'].count(), "entries having 'Price' value greater than", higher)

# Q1 = df['price'].quantile(0.25)
# Q3 = df['price'].quantile(0.75)
# IQR = Q3 - Q1
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR
# df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
# outlier_count = df.shape[0] - df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)].shape[0]
# print(f"Removed {outlier_count} outlier rows from 'price' column.")

# z_scores = stats.zscore(df['price'])
# abs_z_scores = abs(z_scores)
# outliers = (abs_z_scores < 2)  # Keeping data within 2 standard deviations
# df = df[outliers]
# # Display the outliers
# print(f"Number of outliers detected using Z-Score: {outliers.shape[0]}")
# print(outliers)



10211 entries having 'Price' value lower than  10.486511795814852
14958 entries having 'Price' value greater than 14.063110219802613
0 entries having 'Price' value lower than 10.486511795814852
0 entries having 'Price' value greater than 14.063110219802613


### Identifying most frequent price
- .mode[0]: extracts first mode value

In [26]:
spike_value = df['price'].mode()[0]
print(f"Most frequent price: {spike_value}")
print(df[df['price'] == spike_value].shape[0], "rows have this price value.")

Most frequent price: 14.063110219802613
14958 rows have this price value.


### Saving the final cleaned data

In [27]:
df.to_csv('cleaned_subset.csv', index=False)
print("Data cleaning complete. Cleaned file saved as 'cleaned_subset.csv'.")


Data cleaning complete. Cleaned file saved as 'cleaned_subset.csv'.
